In [1]:
import os
path = os.path.join('/home/santiago/Documents/dev/reservoirpy')
import sys
sys.path.insert(0,path)
from reservoirpy.pvtpy import black_oil as bl
from reservoirpy.wellproductivitypy import pi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
ubh = pi.ubh(
    brand='longdi', 
    surf_to_pump_depth=4500,
    pump_to_perf_depth=5820,
    injection_di=1.8,
    return_di=1.19,
    prod_di=2.0,
    nozzle=3.59, 
    throat=6.09
)

In [5]:
issubclass(type(ubh),pi.als)

True

In [6]:
ubh.annulus_area()

0.02946034402972634

In [7]:
ubh.get_area('nozzle')

0.01568958098717009

In [8]:
ubh.get_area('throat')

0.04514992501689643

In [11]:
ubh.pump_to_perf_depth

array([4500.        , 4594.28571429, 4688.57142857, 4782.85714286,
       4877.14285714, 4971.42857143, 5065.71428571, 5160.        ,
       5254.28571429, 5348.57142857, 5442.85714286, 5537.14285714,
       5631.42857143, 5725.71428571, 5820.        ])

In [ ]:
ubh.injection_di